# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import sqlite3
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import PorterStemmer

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

nltk.download(['words', 'punkt', 'stopwords',
               'averaged_perceptron_tagger',
               'maxent_ne_chunker', 'wordnet'])

[nltk_data] Downloading package words to
[nltk_data]     /Users/ayemyatwinshwe/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ayemyatwinshwe/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ayemyatwinshwe/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/ayemyatwinshwe/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/ayemyatwinshwe/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ayemyatwinshwe/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('message_category', engine)

In [3]:
df.head(2)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0


In [4]:
# number of distinct observations
df.nunique()

id                        26180
message                   26177
original                   9630
genre                         3
related                       2
request                       2
offer                         2
aid_related                   2
medical_help                  2
medical_products              2
search_and_rescue             2
security                      2
military                      2
child_alone                   1
water                         2
food                          2
shelter                       2
clothing                      2
money                         2
missing_people                2
refugees                      2
death                         2
other_aid                     2
infrastructure_related        2
transport                     2
buildings                     2
electricity                   2
tools                         2
hospitals                     2
shops                         2
aid_centers                   2
other_in

In [5]:
# number of missing values
df.isnull().sum()

id                            0
message                       0
original                  16045
genre                         0
related                       0
request                       0
offer                         0
aid_related                   0
medical_help                  0
medical_products              0
search_and_rescue             0
security                      0
military                      0
child_alone                   0
water                         0
food                          0
shelter                       0
clothing                      0
money                         0
missing_people                0
refugees                      0
death                         0
other_aid                     0
infrastructure_related        0
transport                     0
buildings                     0
electricity                   0
tools                         0
hospitals                     0
shops                         0
aid_centers                   0
other_in

In [6]:
# drop id, original
df.drop(['id', 'original'], axis=1, inplace=True)

In [7]:
df.head(2)

,message,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,Weather update - a cold front from Cuba that c...,direct,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Is the Hurricane over or is it not over,direct,1,0,0,1,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0


In [8]:
X = df['message'].values
Y = df.loc[:,'related':'direct_report'].values

### 2. Write a tokenization function to process your text data

In [9]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    tokens = word_tokenize(text)

    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()

    clean_tokens = [lemmatizer.lemmatize(tok)
                    for tok in tokens if tok not in stop_words]

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [10]:
pipeline_ada = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize,
                             max_features=None, max_df=1.0, ngram_range=(1, 2))),
    ('tfidf', TfidfTransformer(use_idf=True)),
    ('clf', MultiOutputClassifier(AdaBoostClassifier())),
])

In [11]:
pipeline_ada.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 2), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x1a1b4d4ef0>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R',
                                                      base_estimator=None,
                                                      learning_rate=1.0,
                                                      n_estimators=50,
                                                      random_state=None),
                       

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [12]:
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.3, random_state=42)

In [13]:
pipeline_ada.fit(X_train, Y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 2), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at 0x1a1b4d4ef0>,
                                 vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R',


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [14]:
Y_pred = pipeline_ada.predict(X_test)

In [15]:
(Y_pred == Y_test).mean()

0.9467719149537331

In [16]:
def display_results(y_test, y_pred, y_col):
    
    clf_report = classification_report(y_test, y_pred)
    confusion_mat = confusion_matrix(y_test, y_pred)
    accuracy = (y_pred == y_test).mean() 
    print('\n')   
    print(y_col, ":")
    print('\n')  
    print(clf_report)
    print('confusion_matrix :')
    print(confusion_mat)
    print('\n')
    print('Accuracy =', accuracy)
    print('-'*65)

In [17]:
for i in range(Y_test.shape[1]):
    display_results(Y_test[:, i], Y_pred[:, i],
                    df.loc[:, 'related':'direct_report'].columns[i])



related :


              precision    recall  f1-score   support

           0       0.66      0.26      0.38      1853
           1       0.81      0.96      0.88      6012

    accuracy                           0.79      7865
   macro avg       0.73      0.61      0.63      7865
weighted avg       0.77      0.79      0.76      7865

confusion_matrix :
[[ 486 1367]
 [ 252 5760]]


Accuracy = 0.7941513032422123
-----------------------------------------------------------------


request :


              precision    recall  f1-score   support

           0       0.92      0.96      0.94      6552
           1       0.73      0.56      0.63      1313

    accuracy                           0.89      7865
   macro avg       0.82      0.76      0.79      7865
weighted avg       0.88      0.89      0.89      7865

confusion_matrix :
[[6283  269]
 [ 579  734]]


Accuracy = 0.8921805467260012
-----------------------------------------------------------------


offer :


              prec



hospitals :


              precision    recall  f1-score   support

           0       0.99      1.00      0.99      7791
           1       0.33      0.19      0.24        74

    accuracy                           0.99      7865
   macro avg       0.66      0.59      0.62      7865
weighted avg       0.99      0.99      0.99      7865

confusion_matrix :
[[7763   28]
 [  60   14]]


Accuracy = 0.9888111888111888
-----------------------------------------------------------------


shops :


              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7833
           1       0.00      0.00      0.00        32

    accuracy                           0.99      7865
   macro avg       0.50      0.50      0.50      7865
weighted avg       0.99      0.99      0.99      7865

confusion_matrix :
[[7821   12]
 [  32    0]]


Accuracy = 0.9944055944055944
-----------------------------------------------------------------


aid_centers :


            

### 6. Improve your model
Use grid search to find better parameters. 

In [18]:
%timeit

parameters = {

    # 'vect__max_df': [0.75, 1.0],
    # 'vect__max_features': [None, 5000],
    # 'vect__ngram_range': [(1, 1), (1, 2)],
    # 'tfidf__smooth_idf': [True, False],
    # 'tfidf__sublinear_tf': [True, False],
    # 'tfidf__use_idf': [True, False],
    # 'clf__estimator__learning_rate': [0.5, 1.0],
    'clf__estimator__n_estimators': [20, 50]

}

cv_ada = GridSearchCV(pipeline_ada, param_grid=parameters,
                      cv=2, n_jobs=-1, verbose=2)

cv_ada.fit(X_train, Y_train)

Fitting 2 folds for each of 2 candidates, totalling 4 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:  1.4min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:  1.4min finished


GridSearchCV(cv=2, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 2),
                                                        prep

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [19]:
Y_pred = cv_ada.predict(X_test)

In [20]:
(Y_pred == Y_test).mean()

0.9467507240234513

In [21]:
cv_ada.get_params()

{'cv': 2,
 'error_score': nan,
 'estimator__memory': None,
 'estimator__steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 2), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x1a1b4d4ef0>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R',
                                                      base_estimator=None,
                                                      learning_rate=1.0,
                                                      n_estimators=50,
                                            

In [22]:
for i in range(Y_test.shape[1]):
    display_results(Y_test[:,i], Y_pred[:,i], df.loc[:,'related':'direct_report'].columns[i])



related :


              precision    recall  f1-score   support

           0       0.66      0.26      0.38      1853
           1       0.81      0.96      0.88      6012

    accuracy                           0.79      7865
   macro avg       0.73      0.61      0.63      7865
weighted avg       0.77      0.79      0.76      7865

confusion_matrix :
[[ 486 1367]
 [ 252 5760]]


Accuracy = 0.7941513032422123
-----------------------------------------------------------------


request :


              precision    recall  f1-score   support

           0       0.92      0.96      0.94      6552
           1       0.73      0.56      0.63      1313

    accuracy                           0.89      7865
   macro avg       0.82      0.76      0.79      7865
weighted avg       0.88      0.89      0.89      7865

confusion_matrix :
[[6283  269]
 [ 579  734]]


Accuracy = 0.8921805467260012
-----------------------------------------------------------------


offer :


              prec



hospitals :


              precision    recall  f1-score   support

           0       0.99      1.00      0.99      7791
           1       0.35      0.19      0.25        74

    accuracy                           0.99      7865
   macro avg       0.67      0.59      0.62      7865
weighted avg       0.99      0.99      0.99      7865

confusion_matrix :
[[7765   26]
 [  60   14]]


Accuracy = 0.9890654799745708
-----------------------------------------------------------------


shops :


              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7833
           1       0.00      0.00      0.00        32

    accuracy                           0.99      7865
   macro avg       0.50      0.50      0.50      7865
weighted avg       0.99      0.99      0.99      7865

confusion_matrix :
[[7820   13]
 [  32    0]]


Accuracy = 0.9942784488239034
-----------------------------------------------------------------


aid_centers :


            

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [23]:
# Add a customer transformer

def tokenize_2(text):
    
        detected_urls = re.findall(url_regex, text)
        for url in detected_urls:
            text = text.replace(url, "urlplaceholder")

        tokens = word_tokenize(text)
        lemmatizer = WordNetLemmatizer()
        clean_tokens = [lemmatizer.lemmatize(tok).lower().strip() for tok in tokens]

        return clean_tokens
    
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize_2(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [24]:
pipeline_xgb = Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize,
                                     max_features=None,
                                     max_df=1.0)),
            ('tfidf', TfidfTransformer(use_idf=True))
        ])),

        ('start_verb', StartingVerbExtractor())
    ])),

    ('clf', MultiOutputClassifier(XGBClassifier(
        max_depth=3,
        learning_rate=0.1,
        colsample_bytree=0.7,
        subsample=0.8,
        n_estimators=100,
        min_child_weight=1,
        gamma=5)))
])

In [25]:
pipeline_xgb.fit(X_train, Y_train)

Pipeline(memory=None,
         steps=[('features',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('text_pipeline',
                                                 Pipeline(memory=None,
                                                          steps=[('vect',
                                                                  CountVectorizer(analyzer='word',
                                                                                  binary=False,
                                                                                  decode_error='strict',
                                                                                  dtype=<class 'numpy.int64'>,
                                                                                  encoding='utf-8',
                                                                                  input='content',
                                                                                  low

In [26]:
Y_pred = pipeline_xgb.predict(X_test)

In [27]:
(Y_pred == Y_test).mean()

0.9490146217418944

In [28]:
for i in range(Y_test.shape[1]):
    display_results(Y_test[:, i], Y_pred[:, i],
                    df.loc[:, 'related':'direct_report'].columns[i])



related :


              precision    recall  f1-score   support

           0       0.75      0.16      0.27      1853
           1       0.79      0.98      0.88      6012

    accuracy                           0.79      7865
   macro avg       0.77      0.57      0.57      7865
weighted avg       0.78      0.79      0.73      7865

confusion_matrix :
[[ 300 1553]
 [  99 5913]]


Accuracy = 0.7899554990464082
-----------------------------------------------------------------


request :


              precision    recall  f1-score   support

           0       0.90      0.98      0.94      6552
           1       0.84      0.47      0.60      1313

    accuracy                           0.90      7865
   macro avg       0.87      0.73      0.77      7865
weighted avg       0.89      0.90      0.88      7865

confusion_matrix :
[[6430  122]
 [ 692  621]]


Accuracy = 0.8965034965034965
-----------------------------------------------------------------


offer :


              prec

/Users/ayemyatwinshwe/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))




water :


              precision    recall  f1-score   support

           0       0.98      0.99      0.98      7372
           1       0.79      0.67      0.73       493

    accuracy                           0.97      7865
   macro avg       0.89      0.83      0.85      7865
weighted avg       0.97      0.97      0.97      7865

confusion_matrix :
[[7286   86]
 [ 163  330]]


Accuracy = 0.968340750158932
-----------------------------------------------------------------


food :


              precision    recall  f1-score   support

           0       0.98      0.98      0.98      7004
           1       0.82      0.80      0.81       861

    accuracy                           0.96      7865
   macro avg       0.90      0.89      0.89      7865
weighted avg       0.96      0.96      0.96      7865

confusion_matrix :
[[6855  149]
 [ 175  686]]


Accuracy = 0.9588048315321043
-----------------------------------------------------------------


shelter :


              precisio



other_weather :


              precision    recall  f1-score   support

           0       0.95      0.99      0.97      7463
           1       0.45      0.09      0.15       402

    accuracy                           0.95      7865
   macro avg       0.70      0.54      0.56      7865
weighted avg       0.93      0.95      0.93      7865

confusion_matrix :
[[7420   43]
 [ 367   35]]


Accuracy = 0.9478703115066751
-----------------------------------------------------------------


direct_report :


              precision    recall  f1-score   support

           0       0.86      0.98      0.91      6343
           1       0.78      0.32      0.45      1522

    accuracy                           0.85      7865
   macro avg       0.82      0.65      0.68      7865
weighted avg       0.84      0.85      0.82      7865

confusion_matrix :
[[6204  139]
 [1038  484]]


Accuracy = 0.8503496503496504
-----------------------------------------------------------------


### 9. Export your model as a pickle file

In [29]:
import pickle
pickle.dump(pipeline_xgb,open('./models/model_xgb','wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.